- 가중치와 절편은 데이터에서 학습됩니다
- f는 활성화 함수 , w는 가중치, b는 절편
- y = f(wx + b) (x와 w는 벡터, x와 w의 곱셈은 점곱 dot product)
- 선형 함수 표현인 wx + b를 아핀 변환(affine transform)이라고도 부릅니다.
- f로 표시한 활성화 함수는 일반적으로 비선형 함수
- 퍼셉트론은 선형 함수와 비선형 함수의 조합


In [ ]:
# 코드3-1. 파이토치로 구현한 퍼셉트론

import torch
import torch.nn as nn

# torch.nn은 Linear 클래스를 제공
# 가중치와 절편은 nn.Linear 클래스 안에서 관리됨
# 절편이 필요없는 모델이 필요할 경우, nn.Linear 생성자 호출 시 bias=False로 지정 가능

class Perceptron(nn.Module):
    """퍼셉트론은 하나의 선형 층 입니다"""
    def _init_(self, input_dim):
        """
        매개변수:
            input_dim (int): 입력 특성의 크기
        """
        super(Perceptron, self).__init__()
        self.fc1 = nn.Linear(input_dim, 1)

    def forward(self, x_in):
        """
        퍼셉트론의 정방향 계산

        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, num_features)입니다.
        변환값:
            결과 텐서. tensor.shape는 (batch,)입니다.
        """
        return torch.sigmoid(self.fc1(x_in)).squeeze()




- 활성화 함수는 비선형 함수
- 신경망에서 데이터의 복잡한 관계를 감지하는데 사용함

### 3.2.1 시그모이드

- 시그모이드는 초기 활성화함수
- 임의의 실숫값을 받아 0과 1사이의 범위로 압축함
- 시그모이드는 극단적인 출력을 만듬 (그레이디언트가 0 또는 발산 -> 문제: 그레디이언트 소실 문제라고함: vanishing gradient problem)
- 신경망에서 시그모이드 활성화 함수는 거의 출력층에서만 사용됨
- 출력층에서는 출력을 확률로 압축하는데 시그모이드 함수를 사용함



In [ ]:
# 코드3-2. 시그모이드 활성화 함수

import torch
import matplotlib.pyplot as plt

x = torch.range(-5., 5., 0.1)
y = torch.sigmoid(x)
plt.plot(x.numpy(), y.numpy())
plt.show()


### 3.2.2 하이볼릭 탄젠트

코드3-9. 이진 크로스 엔트로피 손실

In [ ]:
import torch
import torch.nn as nn

bce_loss = nn.BCELoss()
sigmoid = nn.Sigmoid()
probabilities = sigmoid(torch.randn(4, 1, requires_grad=True))
targets = torch.tensor([1, 0, 1, 0], dtype=torch.float32).view(4,1)
loss = bce_loss(probabilities, targets)
print(probabilities)
print(loss)


코드 3-10. Adam 옵티마이저 준비

In [ ]:
import torch.nn as nn
import torch.optim as optim

input_dim = 2
lr = 0.001

perception = Perception(input_dim=input_dim)
bce_loss = nn.BCELoss()
optimizer = optim.Adam(params=perception.parameters(), lr=lr)


코드 3-11. 퍼셉트론과 이진 분류를 위한 지도 학습 훈련 반복

In [ ]:
# 각 에포크는 전체 훈련 데이터를 사용합니다
for epoch_i in range(n_epochs):
    # 내부 반복은 데이터셋에 있는 배치에 대해 수행됩니다
    for batch_i in range(n_batches):

        # 0단계: 데이터 가져오기
        x_data, y_target = get_toy_data(batch_size)

        # 1단계: 그레이디언트 초기화
        perceptron.zero_grad()

        # 2단계: 모델의 정방향 계산 수행하기
        y_pred = perceptron(x_data, apply_sigmoid=True)

        # 3단계: 최적화하려는 손실 계산하기
        loss = bce_loss(y_pred, y_target)

        # 4단계: 손실 신호를 거꾸로 전파하기
        loss.backward()
        # 5단계: 옵티마이저로 업데이트하기
        optimizer.step()



### 3.6.1 옐프 리뷰 데이터셋
코드 3-12. 훈련, 검증, 테스트 세트 만들기

In [ ]:
# 별점 기준으로 나누어 훈련, 검증, 테스트를 만듭니다.
by_rating = collections.defaultdict(list)
for _, row in review_subset.iterrows():
    by_rating[row.rating].append(row.to_dict())

# 분할 데이터를 만듭니다.
final_list = []
np.random.seed(args.seed)

for _, item_list in sorted(by_rating.items()):
    np.random.shuffle(item_list)
    n_total = len(item_list)
    n_train = int(args.train_proportion * n_total)
    n_val = int(args.val_proportion * n_total)
    n_test = int(args.test_proportion * n_total)

    # 데이터 포인터에 분할 속성을 추가합니다
    for item in item_list[:n_train]:
        item['split'] = 'train'

    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'

    for item in item_list[n_train+n_val:n_train+n_val+n_test]:
        item['split'] = 'test'

    # 최종 리스트에 추가합니다
    final_list.extend(item_list)

final_reviews = pd.DataFrame(final_list)

코드 3-13. 최소한의 데이터 정제 작업

In [ ]:
import re

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text) # 구두점 기호 앞뒤에 공백을 넣고
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text) # 구두점이 아닌 기호를 제거하는 데이터 정제 작업을 모든 세트에 수행

final_reviews.review = final_reviews.review.apply(preprocess_text)


코드 3-14. 옐프 리뷰 데이터를 위한 파이토치 데이터셋 클래스

In [ ]:
from torch.utils.data import Dataset

class ReviewDataset(Dataset):
    def __init__(self, review_df, vectorizer):
        """
        매개변수:
            review_df (pandas.DataFrame): 데이터셋
            vectorizer (ReviewVectorizer): ReviewVectorizer 객체
        """
        self.review_df = review_df
        self._vectorizer = vectorizer

        self.train_df = self.review_df[self.review_df.split == 'train']
        self.train_size = len(self.train_df)

        self.val_df = self.review_df[self.review_df.split == 'val']
        self.validation_size = len(self.val_df)

        self.test_df = self.review_df[self.review_df.split == 'test']
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size),
                             'test': (self.test_df, self.test_size)}

        self.set_split('train')

    @classmethod
    def load_dataset_and_make_vectorizer(cls, review_csv):
        """ 데이터셋을 로드하고 새로운 ReviewVectorizer 객체를 만듭니다

        매개변수:
            review_csv (str): 데이터셋의 위치
        반환값:
            ReviewDataset의 인스턴스
        """
        review_df = pd.read(review_csv)
        return cls(review_df, ReviewVectorizer.from_dataframe(review_df))

    def get_vectorizer(self):
        """ ReviewVectorizer 객체를 반환합니다 """
        return self._vectorizer

    def set_split(self, split="train"):
        """ 데이터프레임에 있는 열을 사용해 분할 세트를 선택합니다

        매개변수:
            split (str): "train", "val", "test" 중 하나
        """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """ 파이토치 데이터세세의 주요 진입 메서드

        매개변수:
            index (int): 데이터 포인트의 인덱스
        반환값:
            데이터 포인트의 특성(x_data)과 레이블(y_target)로 이루어진 딕셔너리
        """
        row = self._target_df.iloc[index]

        review_vector = \
            self._vectorizer.vectorize(row.review)

        ratin_index = \
            self._vectorizer.rating_vocab.lookup_token(row.rating)

        return {'x_data': review_vector,
                'y_target': rating_index}

    def get_num_batches(self, batch_size):
        """ 배치 크기가 주어지면 데이터셋으로 만들 수 있는 배치 개수를 반환합니다

        매개변수:
            batch_size (int)
        반환값:
            배치 개수
        """
        return len(self) // batch_size

코드3-15. 머신러닝 파이프라인에 필요한 토큰과 정수 매핑을 관리하는 Vocabulary 클래스

In [ ]:
class Vocabulary(object):
    """ 매핑을 위해 텍스트를 처리하고 어휘 사전을 만드는 클래스 """
    def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):
        """
        매개변수:
            token_to_idx(dict): 기존 토큰-인덱스 매핑 딕셔너리
            add_unk (bool): UNK 토큰을 추가할지 지정하는 플래그
            unk_token (str): Vocabulary에 추가할 UNK 토큰
        """

        if token_to_idx is None:
            token_to_idx = {}
            self._token_to_idx = token_to_idx

            self._idx_to_token = {idx: token
                                  for token, idx in self._token_to_idx.items()}

            self._add_unk = add_unk
            self._unk_token = unk_token

            self.unk_index = -1
            if add_unk:
                self.unk_index = self.add_token(unk_token)

    def to_serializable(self):
        """ 직렬화할 수 있는 딕셔너리를 반환합니다 """
        return {'token_to_idx': self._token_to_idx,
                'add_unk': self._add_unk,
                'unk_token': self._unk_token}

    @classmethod
    def from_serializable(cls, contents):
        """ 직렬화된 딕셔너리에서 Vocabulary 객체를 만듭니다 """
        return cls(**contents)

    def add_token(self, token):
        """ 토큰을 기반으로 매핑 딕셔너리를 업데이트합니다

        매개변수:
            token (str): Vocabulary에 추가할 토큰
        반환값:
            index (int): 토큰에 상응하는 정수
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index

        def lookup_token(self, token):
            """ 토큰에 대응하는 인덱스를 추출합니다.
            토큰이 없으면 UNK 인덱스를 반환합니다.

            매개변수:
                token (str): 찾을 토큰
            반환값:
                index (int): 토큰에 해당하는 인덱스
            노트:
                UNK 토큰을 사용하려면 (Vocabulary에 추가하기 위해)
        'unk_index'가 0보다 커야 합니다.
            """
            if self.add_unk:
                return self._token_to_idx.get(token, self.unk_index)
            else:
                return self._token_to_idx[token]

        def lookup_index(self, index):
            """ 인덱스에 해당하는 토큰을 반환합니다.

            매개변수:
                index (int): 찾을 인덱스
            반환값:
                token (str): 인덱스에 해당하는 토큰
            에러:
                KeyError: 인덱스가 Vocabulary에 없을 때 발생합니다.
            """
            if index not in self._idx_to_token:
                raise KeyError("Vocabulary에 인덱스(%d)가 없습니다." % index)
            return self._idx_to_token[index]

        def __str__(self):
            return "<Vocabulary(size=%d)>" % len(self)

        def __len__(self):
            return len(self._token_to_idx)

